## Import Packages

In [2]:
import os
import glob
import numpy as np
import pandas as pd
#import network_fcon as fc
import scipy as sp
from scipy.stats import pearsonr
from scipy.stats import linregress
import seaborn as sns
import matplotlib.pyplot as plt
import re

### Set Variables and Paths

In [3]:
# Set paths
path = "/Users/pecsok/Desktop/ImageData/PMACS_remote"
path = "/Users/pecsok/Desktop/ImageData/PMACS_remote/analyses/S1000"

# Import group dataframes and set indices
subjlist = pd.read_csv(path + "data/pipeline_input_all_subj.csv", sep=',') 
#cestmat = pd.read_csv("cest_parcelmat" + fieldstrength + ".csv", sep=',') 
#fcmat = pd.read_csv("fc_parcelmat_" + fieldstrength + ".csv", sep=',')
#rehomat = pd.read_csv("reho_parcelmat" + fieldstrength + ".csv", sep=',')
grp_df = pd.read_csv("filtered_grp_df" + fieldstrength + ".csv", sep=',')

# Reformat some dfs
cestmat.set_index('BBLID', inplace = True)
fcmat.set_index('BBLID', inplace = True)
rehomat.set_index('BBLID', inplace = True)
grp_df.set_index('BBLID', inplace = True)
subjlist = subjlist.dropna(how='all')
subjlist.rename(columns={'fMRI Field Strength': 'fieldstrength'},inplace=True)
# Correct subject list for future! 
subjlist.loc[subjlist['BBLID'] == 88608.0, 'SCANID_rs'] = 'motive2'
subjlist.loc[subjlist['BBLID'] == 20792.0, 'SCANID_rs'] = 'motive2'
subjlist.loc[subjlist['BBLID'] == 88760.0, 'SCANID_rs'] = 'motive2'
subjlist.loc[subjlist['BBLID'] == 112126.0, 'SCANID_rs'] = 'motive1'
subjlist.loc[subjlist['BBLID'] == 91962.0, 'SCANID_rs'] = 'motive1'
subjlist.loc[subjlist['BBLID'] == 89279.0, 'SCANID_rs'] = 'motive1'
subjlist.loc[subjlist['BBLID'] == 91422.0, 'SCANID_rs'] = 'motive1'
subjlist.loc[subjlist['BBLID'] == 121407.0, 'SCANID_rs'] = '9257' 

# Set Variables
fieldstrength = '7T'
bblids = []
sesids = []
diag_scores = ["dx_pscat", "hstatus","sex", "age", "race","ethnic"] # ,"axis1_desc1","axis1_stat1"
diag_details = ["axis1_desc1", "axis1_desc2", "axis1_desc3","axis1_desc4","axis1_desc5", "axis1_desc6"]
subjlist = subjlist[subjlist['fieldstrength'] == fieldstrength]
subjs = subjlist['BBLID']


In [ ]:
# Create correlation matrix and heatmap

In [10]:
mean_df = cestmat.mean()
print(mean_df).reset_index()

Unnamed: 0         34.000000
Session         10836.043478
dx_pscat                 NaN
hstatus                  NaN
sex                      NaN
                    ...     
NZcount_998              NaN
NZMean_999               NaN
NZcount_999              NaN
NZMean_1000              NaN
NZcount_1000             NaN
Length: 1008, dtype: float64


AttributeError: 'NoneType' object has no attribute 'reset_index'

In [ ]:
# Create df with parcel-wise averages for fc, reho, GluCEST, and nmaps
for subj in subjs: # Loop through subjs
    bblid = str(int(subj))
    ses=subjlist.loc[subjlist['BBLID'] == float(bblid), 'SCANID_rs'].values[0]
    for parcel in parcels: # Loop through parcels
        col_name = f'NZMean_{i}'
        ct_name = f'NZcount_{i}'
        

In [ ]:
# First, let's make a heatmap 

